Eine Vorlage namens "geo_cross_walk" liegt bereits in "data". Hierhin werden die geocrosswalk-daten, z.B. aus QGIS, eingefügt. Die Spaltennamen, also die Namen der einzelnen Ebenen, können jetzt beliebig benannt werden, aber später NICHT mehr umbenannt. Beliebig viele Ebenen sind möglich. Die Beispieldaten können gelöscht werden.
Daraus wird eine Vorlage für die Controls der untersten Ebene erstellt.

In [1]:
import json
import os

import openpyxl
from unidecode import unidecode

# USER INPUTS
# Change the working directory if necessary
#os.chdir('D:\BSCloud\Masterarbeit Felix\Synthetische Population\_PopSimStarterPack')

# Set this to True if you want to output all intermediate PopSim files
output_everything = False
# Specify the number of processors to use for multiprocessing
# This should be the number of cores on your machine minus 1
# If you don't know how many cores your machine has, you can find out by running:
# print(multiprocessing.cpu_count())
# If you don't want to use multiprocessing, set this to 1
num_processors = 1
# Seed geography should not be changed.
seed_geography = "STAAT"
# END USER INPUTS


existing_workbook = openpyxl.load_workbook('data/_prep0_geo_cross_walk.xlsx')
worksheet = existing_workbook.active
# unidecode(Ä to A etc.) just to be sure
lowest_geography_name = unidecode(worksheet['A1'].value)
worksheet['A1'].value = lowest_geography_name
existing_workbook.close()

output_filename = f'data/_prep1_control_totals_{lowest_geography_name}.xlsx'

if not os.path.exists(output_filename):
    new_workbook = openpyxl.Workbook()
    new_worksheet = new_workbook.active
    new_worksheet['A1'] = lowest_geography_name
    new_worksheet['B1'] = 'numberOfHouseholds'

    new_workbook.save(output_filename)
    new_workbook.close()
    print(f"Created '{output_filename}'")
else:
    print(f"'{output_filename}' already exists, not overwritten.")

# Save vars to a config file
config = {
    "lowest_geography_name": lowest_geography_name,
    "num_processors": num_processors,
    "output_everything": output_everything,
    "seed_geography": seed_geography
}
with open("prep_config.json", "w") as config_file:
    json.dump(config, config_file)

Created 'data/_prep1_control_totals_Gitter_ID_100m.xlsx'


In dieses neue Workbook werden die Control-daten der untersten Ebene eingefügt. Die Anzahl der Haushalte ist eine zwingende Angabe.

Es kann sein, dass die Rohdaten der untersten Ebene nicht alle im geo-cross-walk erstellten Blocks (bzw. Gebäude o.Ä.) abdecken; oder dass es Daten für Blocks gibt die nicht betrachtet werden. Dies wird hier abgeglichen. 
Dann werden Vorlagen für die Controls aller Ebenen erstellt.

In [2]:
import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import os
import json

# Load vars
with open("prep_config.json", "r") as config_file:
    config = json.load(config_file)

lowest_geography_name = config["lowest_geography_name"]

# Read the two Excel worksheets into Pandas DataFrames
df1 = pd.read_excel('data/_prep0_geo_cross_walk.xlsx')
df2 = pd.read_excel(f'data/_prep1_control_totals_{lowest_geography_name}.xlsx')

# Extract the common numbers
common_numbers = set(df1[lowest_geography_name]).intersection(df2[lowest_geography_name])

# Filter the original DataFrames to keep only rows with common numbers
filtered_df1 = df1[df1[lowest_geography_name].isin(common_numbers)]
filtered_df2 = df2[df2[lowest_geography_name].isin(common_numbers)]

#Sort both dataframes by the lowest geography
filtered_df1 = filtered_df1.sort_values(by=[lowest_geography_name]).reset_index(drop=True)
filtered_df2 = filtered_df2.sort_values(by=[lowest_geography_name]).reset_index(drop=True)

print(f"Number of rows in _prep1_geo_cross_walk: {len(df1)}")
print(f"Number of rows in geo_cross_walk: {len(filtered_df1)}")
print(f"removed {len(df1) - len(filtered_df1)} rows")
print(f"Number of rows in _prep1_control_totals_{lowest_geography_name}: {len(df2)}")
print(f"Number of rows in control_totals_{lowest_geography_name}: {len(filtered_df2)}")
print(f"removed {len(df2) - len(filtered_df2)} rows")


def unidecode_and_clean_column_names(df):
    modified_columns = {}
    for unclean_column_name in df.columns:
        # Apply unidecode transformation to convert non-ASCII characters
        modified_name = unidecode(unclean_column_name)
        # Remove empty spaces, periods, and commas
        modified_name = modified_name.replace(" ", "").replace(".", "").replace(",", "")
        modified_columns[unclean_column_name] = modified_name
    df.rename(columns=modified_columns, inplace=True)


unidecode_and_clean_column_names(filtered_df1)
unidecode_and_clean_column_names(filtered_df2)

print(f"Number of rows in geo_cross_walk: {len(filtered_df1)}")
print(f"Number of rows in control_totals_{lowest_geography_name}: {len(filtered_df2)}")

# It shouldn't be possible that the first columns are different, but just to be sure
if not np.array_equal(filtered_df1.iloc[:, 0].values, filtered_df2.iloc[:, 0].values):
    raise ValueError("The values in the first columns are different. Aborting.")

filtered_df1.to_excel('data/geo_cross_walk.xlsx', index=False)
print(f"Created data/geo_cross_walk.xlsx")

# Create control_total templates for all geographies
geo_names = list(filtered_df1.columns)
for i, geo_name in enumerate(geo_names):
    if i == 0:
        continue  # Skip the first column, lowest-geo controls have already been created

    workbook = Workbook()
    worksheet = workbook.active

    # Extract the data for the current column and the columns to the right
    data = filtered_df1.iloc[:, i:]
    data = data.drop_duplicates(subset=geo_name)

    # Write the data to the worksheet
    for row in dataframe_to_rows(data, index=False, header=True):
        worksheet.append(row)
    output_filename = f'data/_prep2_control_totals_{geo_name}.xlsx'
    if not os.path.exists(output_filename):
        workbook.save(output_filename)
        print(f"Created {output_filename}")
    else:
        print(f"{output_filename} already exists, not overwritten.")

# Create finished control_total for lowest geography
filtered_df2 = pd.concat([filtered_df1, filtered_df2.iloc[:, 1:]], axis=1)
for column_name in filtered_df2.columns:
    if column_name not in geo_names:
        new_name = f'{column_name}_{lowest_geography_name}'
        filtered_df2.rename(columns={column_name: new_name}, inplace=True)
filtered_df2.to_excel(f'data/control_totals_{lowest_geography_name}.xlsx', index=False)
print(f"Created data/control_totals_{lowest_geography_name}.xlsx")

# Save vars to the config file
with open("prep_config.json", "r") as config_file:
    existing_config = json.load(config_file)

existing_config["geo_names"] = geo_names

with open("prep_config.json", "w") as config_file:
    json.dump(existing_config, config_file)

Number of rows in _prep1_geo_cross_walk: 22630
Number of rows in geo_cross_walk: 22630
removed 0 rows
Number of rows in _prep1_control_totals_Gitter_ID_100m: 22630
Number of rows in control_totals_Gitter_ID_100m: 22630
removed 0 rows
Number of rows in geo_cross_walk: 22630
Number of rows in control_totals_Gitter_ID_100m: 22630
Created data/geo_cross_walk.xlsx
Created data/_prep2_control_totals_STAAT.xlsx
Created data/_prep2_control_totals_WELT.xlsx
Created data/control_totals_Gitter_ID_100m.xlsx


Now the _prep files for the remaining geographies can be filled with data by the user. They may remain unchanged if no controls exist at that level. The prepared data must not be overwritten.
The settings file is fitted, and the controls file prepared.

In [4]:
import pandas as pd
import os
from unidecode import unidecode
import json
import yaml

# Load vars
with open("prep_config.json", "r") as config_file:
    config = json.load(config_file)

lowest_geography_name = config["lowest_geography_name"]
geo_names = config["geo_names"]
num_processors = config["num_processors"]
output_everything = config["output_everything"]
seed_geography = config["seed_geography"]


def unidecode_and_clean_column_names(df):
    modified_columns = {}
    for unclean_column_name in df.columns:
        # Apply unidecode transformation to convert non-ASCII characters
        modified_name = unidecode(unclean_column_name)
        # Remove empty spaces, periods, and commas
        modified_name = modified_name.replace(" ", "").replace(".", "").replace(",", "")
        modified_columns[unclean_column_name] = modified_name
    df.rename(columns=modified_columns, inplace=True)


# Open all control_total files
control_total_files = [file for file in os.listdir('data') if
                       file.startswith('_prep2_control_totals_') or file.startswith('_prep1_control_totals_')]

# Create a dictionary to store data for the new Excel file
new_data = {
    'target': [],
    'geography': [],
    'seed_table': [],
    'importance': [],
    'control_field': [],
    'expression': []
}

# Ensure that the number of opened files matches the number of geographies
if len(control_total_files) != len(geo_names):
    raise ValueError("The number of control_total files does not match the number of geographies. Aborting.")

# Make all non-geo headers unique (this is necessary for PopSim)
for file in control_total_files:
    df = pd.read_excel(f'data/{file}')
    unidecode_and_clean_column_names(df)

    # Extract the geography name from the file name
    geography_name = df.columns[0]
    parts = file.split('_')
    if len(parts) > 3:
        # geography_name = parts[4].split('.')[0]
        # The geography name should match the first value in the first row
        if geography_name != df.columns[0] or geography_name not in geo_names:
            raise ValueError("File or file name does not match the expected format. Aborting.")
    else:
        raise ValueError("File name does not match the expected format. Aborting.")

    # Append the geography name as a postfix to columns that aren't geography names
    for column_name in df.columns:
        if column_name not in geo_names:
            new_name = f'{column_name}_{geography_name}'
            df.rename(columns={column_name: new_name}, inplace=True)

            if new_name.startswith("numberOfHouseholds") and geography_name == lowest_geography_name:
                total_hh_control = f'{new_name}_target'
            new_data['target'].append(f'{new_name}_target')
            new_data['geography'].append(geography_name)
            new_data['control_field'].append(new_name)

    #Don't overwrite the _prep1_ file, it has already been saved as control_totals_{lowest_geography_name}.xlsx
    if not file.startswith('_prep1_'):
        df.to_excel(f'data/{file.replace("_prep2_", "")}', index=False)
        print(f"Created {file.replace('_prep2_', '')}")

if total_hh_control is None:
    raise ValueError("Could not find a total_hh_control. Aborting.")

# Insert example data
new_data['expression'].append('(households.H_GEW > 0) & (households.H_GEW < np.inf)')

# Create a DataFrame from the new_data dictionary
df = pd.DataFrame({key: pd.Series(value) for key, value in new_data.items()})

output_file_name = os.path.join('configs', '_prep3_controls.xlsx')
if not os.path.exists(output_file_name):
    df.to_excel(output_file_name, index=False)
    print(f"Created controls file template '{output_file_name}'.")
else:
    print(f"Controls file template '{output_file_name}' already exists, not overwritten.")

# Edit the yaml settings file ----------------------------------------------
popsim_settings_file_name = os.path.join('configs', 'settings.yaml')
with open(popsim_settings_file_name, "r") as yaml_file:
    popsim_settings = yaml.safe_load(yaml_file)

# Geographies in geo_name are in order from lowest to highest, but PopSim expects them here in top-down order
topdown_geos = geo_names[::-1]
popsim_settings["geographies"] = topdown_geos
popsim_settings["seed_geography"] = seed_geography
popsim_settings["total_hh_control"] = total_hh_control

# Remove all input_tables after "geo_cross_walk"
index_to_remove_from = None
for i, table in enumerate(popsim_settings["input_table_list"]):
    if table["tablename"] == "geo_cross_walk":
        index_to_remove_from = i

if index_to_remove_from is not None:
    popsim_settings["input_table_list"] = popsim_settings["input_table_list"][:index_to_remove_from + 1]
else:
    raise ValueError("Could not find 'geo_cross_walk' in the input_table_list. Aborting.")

# Add new input_tables based on geo_names
for geo_name in geo_names:
    tablename = f"{geo_name}_control_data"
    filename = f"control_totals_{geo_name}.csv"
    new_table_entry = {
        "tablename": tablename,
        "filename": filename
    }
    popsim_settings["input_table_list"].append(new_table_entry)

# Edit output tables
if output_everything:
    popsim_settings["output_tables"]["action"] = "skip"
    #Can't keep it entirely empty, because PopSim will throw an error
    popsim_settings["output_tables"]["tables"] = "geo_cross_walk"
else:
    popsim_settings["output_tables"]["action"] = "include"
    popsim_settings["output_tables"]["tables"] = []
    popsim_settings["output_tables"]["tables"].append("expanded_household_ids")

    for geo_name in geo_names:
        popsim_settings["output_tables"]["tables"].append(f"summary_{geo_name}")
    for geo_name in geo_names[:-2]:  #don't include the two highest geographies (seed and meta)
        popsim_settings["output_tables"]["tables"].append(f"summary_{geo_name}_{seed_geography}")

# Edit models
popsim_settings["models"] = [model for model in popsim_settings["models"] if "sub_balancing" not in model]

integerize_index = popsim_settings["models"].index("integerize_final_seed_weights")

for geo_name in geo_names[:-2]:
    sub_balancing_entry = f"sub_balancing.geography={geo_name}"
    popsim_settings["models"].insert(integerize_index + 1, sub_balancing_entry)

# Save the modified YAML data back to the file
with open(popsim_settings_file_name, "w") as yaml_file:
    yaml.dump(popsim_settings, yaml_file, default_flow_style=False)
print("Modified settings.yaml file.")

# Edit the analysis notebook settings file ----------------------------------------------
analysis_settings_file_name = os.path.join('scripts', 'verification.yaml')
with open(analysis_settings_file_name, "r") as yaml_file:
    analysis_settings = yaml.safe_load(yaml_file)

analysis_settings["group_geographies"] = topdown_geos
try:
    this_folder_name = os.path.basename(os.path.dirname(os.path.abspath(__file__)))
except Exception:
    #if this script is run from the anaconda console, the above line will throw an error
    this_folder_name = os.path.basename(os.getcwd())
analysis_settings["region"] = this_folder_name

analysis_settings["seed_cols"]["geog"] = seed_geography
analysis_settings["summaries"] = []
for geo_name in geo_names:
    analysis_settings["summaries"].append(f"output/final_summary_{geo_name}.csv")
for geo_name in geo_names[:-2]:
    analysis_settings["summaries"].append(f"output/final_summary_{geo_name}_{seed_geography}.csv")

analysis_settings["aggregate_summaries"] = []
for target, geography, control_field in zip(new_data['target'], new_data['geography'], new_data['control_field']):
    entry = {
        'name': control_field,
        'geography': geography,
        'control': f'{target}_control',
        'result': f'{target}_result'
    }
    analysis_settings["aggregate_summaries"].append(entry)


with open(analysis_settings_file_name, "w") as yaml_file:
    yaml.dump(analysis_settings, yaml_file, default_flow_style=False)
print("Modified verification.yaml file.")


Created control_totals_STAAT.xlsx
Created control_totals_WELT.xlsx
Created controls file template 'configs\_prep3_controls.xlsx'.
Modified settings.yaml file.
Modified verification.yaml file.


The User now has to edit the auto-prepared controls.xlsx file in config. A reference from filled projects might help. The script will then filter the seed files accordingly and check for implausibilities.


In [5]:
import pandas as pd
import os
import json
import re
import math

# Load stuff
with open("prep_config.json", "r") as config_file:
    config = json.load(config_file)

controls_file_name = os.path.join('configs', '_prep3_controls.xlsx')
controls_df = pd.read_excel(controls_file_name)

seed_persons_df = pd.read_csv('data/_prep3_seed_persons.csv')
seed_households_df = pd.read_csv('data/_prep3_seed_households.csv')

print(f'Columns of seed_persons: {seed_persons_df.columns}')
print(f'Columns of seed_households: {seed_households_df.columns}')

# Initialize the set with predefined column names that must always be included
column_names_to_include = {'H_ID', 'H_GEW', 'HP_ID', 'P_ID', 'P_GEW'}
unique_column_names = column_names_to_include.copy()

# Define a regular expression pattern to extract column names after the first "."
pattern = r'\.(?P<column_name>[^ ]+)'

for expression in controls_df['expression']:
    if pd.isna(expression) or (isinstance(expression, float) and math.isnan(expression)):
        raise ValueError(
            f"Empty or NaN expression encountered. Make sure to write expressions for all targets in {controls_file_name} and rerun. Aborting.")

    matches = re.finditer(pattern, expression)
    for match in matches:
        column_name = match.group('column_name')
        unique_column_names.add(column_name)

# Filter seed_persons_df and seed_households_df to keep only the unique column names
seed_persons_df = seed_persons_df[list(unique_column_names.intersection(seed_persons_df.columns))]
seed_households_df = seed_households_df[list(unique_column_names.intersection(seed_households_df.columns))]

print(f'Columns of seed_persons after filtering: {seed_persons_df.columns}')
print(f'Columns of seed_households after filtering: {seed_households_df.columns}')

seed_persons_df['STAAT'] = 1
seed_households_df['STAAT'] = 1

# Save the filtered DataFrames to new files
seed_persons_df.to_csv('data/seed_persons.csv', index=False)
print("Created data/seed_persons.csv")
seed_households_df.to_csv('data/seed_households.csv', index=False)
print("Created data/seed_households.csv")
controls_df.to_excel('configs/controls.xlsx', index=False)
print("Created configs/controls.xlsx")


Columns of seed_persons: Index(['HP_ID', 'H_ID', 'P_ID', 'P_GEW', 'P_HOCH', 'MODE', 'MODE_HH',
       'BASISAUF', 'PROXY', 'PROXY_01',
       ...
       'bus28', 'tram28', 'bahn28', 'min_ozmz', 'haustyp', 'hausnutz',
       'garage', 'wohnlage', 'quali_nv', 'quali_opnv'],
      dtype='object', length=201)
Columns of seed_households: Index(['H_ID', 'H_GEW', 'H_HOCH', 'MODE', 'BASISAUF', 'TEILSTP', 'M_CAR',
       'H_ART', 'H_GR', 'hhgr_gr',
       ...
       'bus28', 'tram28', 'bahn28', 'min_ozmz', 'haustyp', 'hausnutz',
       'garage', 'wohnlage', 'quali_nv', 'quali_opnv'],
      dtype='object', length=112)
Columns of seed_persons after filtering: Index(['HP_ALTER', 'P_GEW', 'HP_SEX', 'H_ID', 'HP_ID', 'P_ID'], dtype='object')
Columns of seed_households after filtering: Index(['H_GEW', 'H_ID'], dtype='object')
Created data/seed_persons.csv
Created data/seed_households.csv
Created configs/controls.xlsx


Die Dateien wurden als .xlsx ausgegeben, damit man sie nochmal leicht prüfen kann. Allerdings sollten manuelle Bearbeitungen in diesem Schritt nie nötig sein und können Fehler verursachen. Sieht alles gut aus, konvertiert das Skript alle erstellten control.xlsx in korrekte .csv.

In [6]:
import pandas as pd


def convert_excel_to_csv(folders, prefix):  # intentionally doesn't convert files in subfolders
    for folder in folders:
        control_files = [file for file in os.listdir(folder) if file.startswith(prefix) and file.endswith('.xlsx')]

        for file in control_files:
            file_path = os.path.join(folder, file)
            df = pd.read_excel(file_path)
            csv_file = os.path.splitext(file)[0] + '.csv'
            df.to_csv(os.path.join(folder, csv_file), index=False)
            print(f"Converted {file_path} to {csv_file}")


folders_to_process = ['data', 'configs', 'configs_mp']

#convert all .xlsx with these prefixes to .csv
convert_excel_to_csv(folders_to_process, 'control')
convert_excel_to_csv(folders_to_process, 'geo')



Converted data\control_totals_Gitter_ID_100m.xlsx to control_totals_Gitter_ID_100m.csv
Converted data\control_totals_Gitter_ID_100mBU.xlsx to control_totals_Gitter_ID_100mBU.csv
Converted data\control_totals_STAAT.xlsx to control_totals_STAAT.csv
Converted data\control_totals_WELT.xlsx to control_totals_WELT.csv
Converted configs\controls.xlsx to controls.csv
Converted data\geo_cross_walk.xlsx to geo_cross_walk.csv


Clean up all unnecessary files that were created only as intermediate steps for verification (optional):

In [11]:
import os


def delete_files(folders, prefix, extension):  # intentionally doesn't delete files in subfolders
    for folder in folders:
        if os.path.exists(folder):
            files_in_folder = [file for file in os.listdir(folder) if
                               file.startswith(prefix) and file.endswith(extension)]
            for file in files_in_folder:
                file_path = os.path.join(folder, file)
                try:
                    os.remove(file_path)
                    print(f"Deleted: {file_path}")
                except OSError as e:
                    print(f"Error deleting {file_path}: {e}")
        else:
            print(f"Folder not found: {folder}")


folders_to_process = ['data', 'configs', 'configs_mp']
file_extension = '.xlsx'

delete_files(folders_to_process, 'control', file_extension)
delete_files(folders_to_process, 'geo', file_extension)
#delete_files(folders_to_process, '_', file_extension) #delete all prep files too


Deleted: data\control_totals_BAUBLOCKNR.xlsx
Deleted: data\control_totals_STAAT.xlsx
Deleted: data\control_totals_STADTTLNR.xlsx
Deleted: data\control_totals_WELT.xlsx
Deleted: configs\controls.xlsx
Deleted: data\geo_cross_walk.xlsx


Use in anaconda console in the same directory:
conda activate popsim
python run_populationsim.py